# この notebook を読み進める前に
TeX 出力のためには次のコードを読み込ませること.

In [1]:
from sympy import *
from IPython.display import display
init_printing(use_unicode=True)

 # シンボル定義
a, b, x, y, z, t = symbols('a b x y z t')
x0 = symbols('x_0')
M, N = symbols('M N')
f, g, h = symbols('f g h', cls=Function)

# 式の木構造
まず Sympy で式がどのように表示されるのかを理解する必要がある.
数学の式は木として表現される.
式 $x^2 + xy$ を見てみよう: つまり `x ** 2 + x * y`.
この式は `srepr` を使うと内部的にどう表現されているかわかる.

In [18]:
from sympy import *
x, y, z = symbols('x y z')
expr = x**2 + x*y
display(srepr(expr))

"Add(Pow(Symbol('x'), Integer(2)), Mul(Symbol('x'), Symbol('y')))"

これを実際に木で表示すると見やすくなる (が省略する).
Graphviz と Sympy の `dotprint()` を使うとダイアグラムが簡単に描けるようだ.
あとで試したい.
ちなみに `dotprint()` は[ここ](http://docs.sympy.org/latest/modules/printing.html#sympy.printing.dot.dotprint)参照.

まず木の葉を見てみる.
シンボルはクラス `Symbol` のインスタンスだ.
シンボル定義は次のどちらでもいい.

In [5]:
x = symbols('x')
x = Symbol('x')

この notebook の最初の定義式のように,
複数のシンボルを一気に定義できるので実際のコーディングで使うときはもちろん前者の方が楽.

式の中の数 2 は `Integer(2)` だ.
`Integer` は Sympy での整数に対するクラスで,
Python 標準の int 型ではなく Sympy の中で協調してうまく動く Sympy の型だ.

`x**2` は `Pow` オブジェクトを作る.

In [17]:
display(srepr(x**2))

"Pow(Symbol('x'), Integer(2))"

同じオブジェクトは `Pow(x, 2)` でも作れる.

In [16]:
display(Pow(x, 2))
display(srepr(Pow(x, 2)))

"Pow(Symbol('x'), Integer(2))"

## 注意
Python の整数 2 を使っていたとしても,
何らかの形で Sympy と絡ませるとき Sympy のオブジェクトでなくても Sympy のオブジェクトに変換されるようになっている.
これには `sympify()` メソッドが使われている.

In [9]:
print(type(2))
print(type(sympify(2)))

<class 'int'>
<class 'sympy.core.numbers.Integer'>


`x*y` がどう表現されるか見てみる.
想像通り $x$ と $y$ の積になる.
積に対する Sympy のクラスは `Mul` だ.
同じオブジェクトを `Mul(x, y)` でも作れる.

In [11]:
display(srepr(x*y))
display(Mul(x, y))

"Mul(Symbol('x'), Symbol('y'))"

ようやく最終的な式 `x ** 2 + x * y` に辿りついた.
これは 2 つのオブジェクト `Pow(x, 2)` と `Mul(x, y)` の和だ.
Sympy で和に対応するクラスは `Add` でもちろん `Add(Pow(x, 2), Mul(x, y))` で作れる.

In [13]:
display(Add(Pow(x, 2), Mul(x, y)))
display(x**2 + x*y)

もっと大きい木ができる例を出しておく.

In [15]:
expr = sin(x*y)/2 - x**2 + 1/y
display(srepr(expr))

"Add(Mul(Integer(-1), Pow(Symbol('x'), Integer(2))), Mul(Rational(1, 2), sin(Mul(Symbol('x'), Symbol('y')))), Pow(Symbol('y'), Integer(-1)))"

これについてもいくつか追加の説明をしておく.
Sympy には差に関するクラスはない.
`x - y` は `x + -y`, さらに正確には `x + -1*y` で表現する: `Add(x, Mul(-1, y))`.

In [20]:
expr = x - y
display(srepr(x - y))

"Add(Symbol('x'), Mul(Integer(-1), Symbol('y')))"

`1/y` を見てみよう.
差と同じく商にも対応する `Div(1,y)` はない.
実際には `Pow(y, -1)` と `Pow()` で表現する.

In [21]:
expr = x/y
display(srepr(expr))

"Mul(Symbol('x'), Pow(Symbol('y'), Integer(-1)))"

`x/y` は `x * y ** -1` と表現する: つまり `Mul(x, Pow(y, -1))`.

`sin(x*y)/2` を見てみる.
`Mul(sin(x*y), Pow(Integer(2), -1))` になると思うかもしれない.
しかし実際には `Mul(Rational(1, 2), sin(x*y))` になる.
有理数は積の中では常に 1 つの項にまとめられる.
だから `2` で割ることは `1/2` をかけると表現される.

最後に 1 つ.
式に出てくる順序と `srepr` や木のグラフで出てくる順番は違う.

In [24]:
print('1 + x の出力結果')
display(1 + x)

1 + x の出力結果


Sympy では `Add` や `Mul` のように可換な演算の引数は適切な順序で保存されていて,
入力された順序とは独立になっている.

### 注意
行列のような非可換な積を考えているなら入力の順番は保存される.
非可換なシンボルも導入できる: `Symbol('A', commutative=False)`.

## まとめ
Sympy の式の木を考えるときに覚えておくべきことは次のようにまとまる:
式の内的な表現と出力法は一致している必要はない.
同じことは入力形にもあてはまる.
ある式操作アルゴリズムで期待通りの結果にならないとき,
それはオブジェクトの内部表現形式と自分の想像が食い違っているときだ.

# 式の表現を通じた再帰

Sympy の全てのオブジェクトは 2 つの重要な attribute がある: `func` と `args`.
## `func`
「`func` はオブジェクトの head だ」という英語の解説を読んだ: この head はどう訳すといいのだろうか.
とりあえずそういうものらしい.
例としては `(x*y).func` は `Mul`.
ふつうはオブジェクトのクラスと同じ.

In [2]:
print((x*y).func)

<class 'sympy.core.mul.Mul'>


`func` についての注意を 2 つ.
1 つはオブジェクトのクラスはそれを作るために使ったオブジェクトのクラスと一致する必要がないこと.
例を見るのが早い.

In [5]:
expr = Add(x, x)
print(expr)
print(expr.func)

2*x
<class 'sympy.core.mul.Mul'>


上で見た通り `expr` が `2*x` なので `Add` ではない.
`Add(x, x)` が `Mul(2, x) に変換されているから.
Sympy のクラスは `__new__` クラスコンストラクタをよく使っていて,
これは __init__ と違ってコンストラクタから返されるクラスが違ってもいい.

もう 1 つは効率の観点からの特例がある.
例えば疎行列ではバカスカ出てくるし, `One` や `Zero` はシングルトンとして扱っている.
シングルトンなら `is` で素早く比較もできる.

In [6]:
print(Integer(2).func)
print(Integer(0).func)
print(Integer(-1).func)

<class 'sympy.core.numbers.Integer'>
<class 'sympy.core.numbers.Zero'>
<class 'sympy.core.numbers.NegativeOne'>


## args
`args` はオブジェクトのトップレベルの引数だ.
`(x*y).args` は `(x, y)` になると思うだろう.
複雑な式になると少し変わってくる.

In [7]:
print((x*y).args)

expr = 3*y**2*x
print(expr.func)
print(expr.args)

(x, y)
<class 'sympy.core.mul.Mul'>
(3, x, y**2)


`expr == Mul(3, y**2, x)` になることがわかる.
実際には `func` と `args` から `expr` は完全に再構成されうることがわかる.
これも具体的に見た方が早い.

In [9]:
expr = 3*y**2*x
print(expr.func(*expr.args))
print(expr == expr.func(*expr.args))

3*x*y**2
True


`Mul` では `Rational` の係数が必ず引数の最初にくる.
しかしそれ以外の全ては特殊なパターンはない.
もちろん適切な順序はある.

In [11]:
expr = y**2*3*x
print(expr.args)

(3, x, y**2)


`Mul` の引数はソートされていて同じ `Mul` からは同じ `args` ができる.
しかしこのソートはソートを一意かつ効率的にするものの数学的な意義を持たないある基準に基づいている.
いまの `expr` の `srepr` 形式は `Mul(3, x, Pow(y, 2))` だ.

In [12]:
expr = y**2*3*x
print(srepr(expr = y**2*3*x))

Mul(Integer(3), Symbol('x'), Pow(Symbol('y'), Integer(2)))


ここで `Pow(y, 2)` の引数を得るにはどうすればいいだろうか?
`y**2` は `expr.args` の第 3 スロットにあることを使えばいい.

In [13]:
expr = y**2*3*x
print(expr.args[2])
print(expr.args[2].args)

y**2
(y, 2)


もう少し深く調べてみよう.

In [15]:
print(y.args)
print(Integer(2).args)

()
()


両方とも引数は空.
Sympy では空引数は式の木の葉にヒットしたことを意味する.

Sympy の式に対しては 2 つの可能性がある.
1 つは引数が空であること: これは式の木の葉のノードであることを意味する.
もう 1 つは引数を持つこと: これは式の木のブランチノード.
引数を持つときは `func` と `args` から完全に再構築できる.
これは key invariant で表現される.

## Key Invariant
全ての適切な Sympy の式は引数を持たないか `expr == expr.func(*expr.args)` を満たさなければならない.
これのおかげで式の木を調べたり変更したり再構成して新しい式を作るシンプルなアルゴリズムを書けるようになる.

これをもとに式の木をどのように再帰的に呼び出せるか調べてみよう.
`args` のネストした構造は再帰関数に完璧に適合する.
ベースケースは空の `args` だ.
式を調べつくし各レベルの引数を出力するシンプルな関数を作ってみよう.

In [16]:
def pre(expr):
    print(expr)
    for arg in expr.args:
        pre(arg)

これが `()` はどうやって式の木の葉をうまく表しているかを見てみよう.
再帰のためにベースケースを書く必要すらない: それは `for` で自動的に処理されている.
関数をテストしてみよう.

In [17]:
expr = x*y + 1
pre(expr)

x*y + 1
1
x*y
x
y


これは pre-order で式の木をなめていく関数だ.
Sympy ではよくある処理なので generator 関数として `preorder_traversal()` と `postorder_traversal()` が既にある.

In [19]:
expr = x*y + 1
for arg in preorder_traversal(expr):
    print(arg)

x*y + 1
1
x*y
x
y


In [20]:
expr = x*y + 1
for arg in postorder_traversal(expr):
    print(arg)

1
x
y
x*y
x*y + 1
